In [1]:
import os
import pandas as pd
import numpy as np
import re
from datetime import date, datetime, timedelta
import warnings
import glob 
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Desactiva las advertencias
warnings.filterwarnings('ignore')

In [2]:
# Definir las columnas de códigos y de diagnósticos en texto libre
columnas_codigos = ['COD_DIAGNOSTICO1', 'COD_DIAGNOSTICO2', 'COD_DIAGNOSTICO3',
                    'COD_DIAGNOSTICO4', 'COD_DIAGNOSTICO5', 'COD_DIAGNOSTICO6',
                    'COD_DIAGNOSTICO7', 'COD_DIAGNOSTICO8', 'COD_DIAGNOSTICO9',
                    'COD_DIAGNOSTICO10']
columnas_diagnosticos = ['DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3',
                         'DIAGNOSTICO4', 'DIAGNOSTICO5', 'DIAGNOSTICO6',
                         'DIAGNOSTICO7', 'DIAGNOSTICO8', 'DIAGNOSTICO9',
                         'DIAGNOSTICO10']


In [12]:
def cargar_dataframe_ambulatorio(ruta):
    encoding = 'ISO-8859-1'
    delimitador = ';'
    return pd.read_csv(ruta, delimiter=delimitador, encoding=encoding, on_bad_lines='skip')

def cargar_dataframe_ieeh(ruta):
    encoding = 'ISO-8859-1'
    delimitador = ';'
    return pd.read_csv(ruta, delimiter=delimitador, encoding=encoding, on_bad_lines='skip')

def filtro_dataframe_ca_tiroides(dataframe):

    # Se crea un patrón regex para identificar los códigos CIE-! asociados al cáncer de próstata
    patron_cancer_prostata = r'^C61'

    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de próstata
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_prostata, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de próstata en texto libre
    patron_diagnostico_prostata_1 = re.compile(r'\bc(?!oer).*pr[oó]stat[aá].*\b', re.IGNORECASE)
    patron_diagnostico_prostata_2 = re.compile(r'\bc[aá]ncer.*pr[oó]stat[aá].*', re.IGNORECASE)
    patron_diagnostico_prostata_3 = re.compile(r'\btumor.*maligno.*pr[oó]stat[aá].*', re.IGNORECASE)
    patron_diagnostico_prostata_4 = re.compile(r'\bcarcinoma.*pr[oó]stat[aá].*', re.IGNORECASE)
    patron_diagnostico_prostata_5 = re.compile(r'\bneoplasia.*pr[oó]stat[aá].*', re.IGNORECASE)
    patron_diagnostico_prostata_6 = re.compile(r'\badenocarcinoma.*pr[oó]stat[aá].*', re.IGNORECASE)
    patron_diagnostico_prostata_7 = re.compile(r'\btumor.*prostatic[aá].**', re.IGNORECASE)
    patron_diagnostico_prostata_8 = re.compile(r'\bcarci.*noma.*glandula.*prostatic[aá]*', re.IGNORECASE)
    patron_diagnostico_prostata_9 = re.compile(r'\btumor.*comportamiento.*(incierto|\bdesconocido).*(pr[oó]stata|prost[aá]tico)\b', re.IGNORECASE)
    patron_diagnostico_prostata_10 = re.compile(r'\b(prost[aá]tico|prost[aá]tica|pr[oó]stata)\s(in\ssitu)\b', re.IGNORECASE)
 

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de próstata en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_prostata_1, na=False) |
        x.str.contains(patron_diagnostico_prostata_2, na=False) |
        x.str.contains(patron_diagnostico_prostata_3, na=False) |
        x.str.contains(patron_diagnostico_prostata_4, na=False) |
        x.str.contains(patron_diagnostico_prostata_5, na=False) |
        x.str.contains(patron_diagnostico_prostata_6, na=False) |
        x.str.contains(patron_diagnostico_prostata_7, na=False) |
        x.str.contains(patron_diagnostico_prostata_8, na=False)).any(axis=1)



    


    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    return dataframe[filtro_codigos | filtro_diagnosticos].copy()

def filtro_dataframe_ca_tiroides_ieeh(dataframe):

    # Definir las columnas de códigos y de diagnósticos en texto libre

    columnas_diagnosticos = ['DIAG1','DIAG2','DIAG3']

    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    
    # Llenar los valores NaN con una cadena vacía en las columnas de diagnóstico
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].fillna('')


    # Patrones para identificar diagnósticos de cáncer de próstata en texto libre
    patron_diagnostico_prostata_1 = re.compile(r'\bc(?!o)[a-zá-ú]*[ \.]*pr[oó]stat[aá][a-zá-ú]*\b', re.IGNORECASE)
    patron_diagnostico_prostata_2 = re.compile(r'\bc[aá]ncer.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_3 = re.compile(r'\btumor.*maligno.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_4 = re.compile(r'\bcarci.*noma.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_5 = re.compile(r'\bneoplasia.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_6 = re.compile(r'\badenocarcinoma.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_7 = re.compile(r'\btumor.*prostatic[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_8 = re.compile(r'\bcarci.*noma.*glandula.*prostatic[aá][a-zá-ú]*', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de próstata en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_prostata_1, na=False) |
        x.str.contains(patron_diagnostico_prostata_2, na=False) |
        x.str.contains(patron_diagnostico_prostata_3, na=False) |
        x.str.contains(patron_diagnostico_prostata_4, na=False) |
        x.str.contains(patron_diagnostico_prostata_5, na=False) |
        x.str.contains(patron_diagnostico_prostata_6, na=False) |
        x.str.contains(patron_diagnostico_prostata_7, na=False) |
        x.str.contains(patron_diagnostico_prostata_8, na=False)).any(axis=1)


    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_tiroides_urgencias(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['CODDIAGNOSTICO']
    columnas_diagnosticos = ['DESCDIAGNOSTICO']

    # Se crea un patrón regex para identificar los códigos CIE-! asociados al cáncer de próstata
    patron_cancer_prostata = r'^C61'

    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de próstata
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_prostata, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de próstata en texto libre
    patron_diagnostico_prostata_1 = re.compile(r'\bc(?!o)[a-zá-ú]*[ \.]*pr[oó]stat[aá][a-zá-ú]*\b', re.IGNORECASE)
    patron_diagnostico_prostata_2 = re.compile(r'\bc[aá]ncer.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_3 = re.compile(r'\btumor.*maligno.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_4 = re.compile(r'\bcarci.*noma.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_5 = re.compile(r'\bneoplasia.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_6 = re.compile(r'\badenocarcinoma.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_7 = re.compile(r'\btumor.*prostatic[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_8 = re.compile(r'\bcarci.*noma.*glandula.*prostatic[aá][a-zá-ú]*', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de próstata en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_prostata_1, na=False) |
        x.str.contains(patron_diagnostico_prostata_2, na=False) |
        x.str.contains(patron_diagnostico_prostata_3, na=False) |
        x.str.contains(patron_diagnostico_prostata_4, na=False) |
        x.str.contains(patron_diagnostico_prostata_5, na=False) |
        x.str.contains(patron_diagnostico_prostata_6, na=False) |
        x.str.contains(patron_diagnostico_prostata_7, na=False) |
        x.str.contains(patron_diagnostico_prostata_8, na=False)).any(axis=1)



    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_codigos | filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_tiroides_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Se crea un patrón regex para identificar los códigos CIE-! asociados al cáncer de próstata
    patron_cancer_prostata = r'^C61'

    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer de próstata
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_prostata, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer de próstata en texto libre
    patron_diagnostico_prostata_1 = re.compile(r'\bc(?!o)[a-zá-ú]*[ \.]*pr[oó]stat[aá][a-zá-ú]*\b', re.IGNORECASE)
    patron_diagnostico_prostata_2 = re.compile(r'\bc[aá]ncer.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_3 = re.compile(r'\btumor.*maligno.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_4 = re.compile(r'\bcarci.*noma.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_5 = re.compile(r'\bneoplasia.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_6 = re.compile(r'\badenocarcinoma.*pr[oó]stat[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_7 = re.compile(r'\btumor.*prostatic[aá][a-zá-ú]*', re.IGNORECASE)
    patron_diagnostico_prostata_8 = re.compile(r'\bcarci.*noma.*glandula.*prostatic[aá][a-zá-ú]*', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer de próstata en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_prostata_1, na=False) |
        x.str.contains(patron_diagnostico_prostata_2, na=False) |
        x.str.contains(patron_diagnostico_prostata_3, na=False) |
        x.str.contains(patron_diagnostico_prostata_4, na=False) |
        x.str.contains(patron_diagnostico_prostata_5, na=False) |
        x.str.contains(patron_diagnostico_prostata_6, na=False) |
        x.str.contains(patron_diagnostico_prostata_7, na=False) |
        x.str.contains(patron_diagnostico_prostata_8, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos)].copy()
    return resultado_filtrado

def agregar_dv(resultado_filtrado):
    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    #condicion = (resultado_filtrado['TIPO_IDENTIFICADOR_PACIENTE'] == 'RUN')
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE
    resultado_filtrado['DV'] = resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'].str[-1]

    #resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
    resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'].str[:-1]

    return resultado_filtrado

def agregar_dv_urgencias(resultado_filtrado):

    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE
    resultado_filtrado['DV'] = resultado_filtrado['RUT'].str[-1]

    resultado_filtrado['RUT'] = resultado_filtrado['RUT'].str[:-1]

    return resultado_filtrado

def agregar_dv_inter(resultado_filtrado):
    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE

    resultado_filtrado['DV'] = resultado_filtrado['NRO_IDENTIFICADOR'].str[-1]

    resultado_filtrado['NRO_IDENTIFICADOR'] = resultado_filtrado['NRO_IDENTIFICADOR'].str[:-1]

    return resultado_filtrado

def obtener_diagnosticos_y_codigos(df, nro_identificacion):
    df_paciente = df[df['NRO_IDENTIFICADOR_PACIENTE'] == nro_identificacion]
    diagnosticos = df_paciente[['DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3', 'DIAGNOSTICO4',
                                'DIAGNOSTICO5', 'DIAGNOSTICO6', 'DIAGNOSTICO7','DIAGNOSTICO8',
                                'DIAGNOSTICO9','DIAGNOSTICO10']].values.flatten()
    codigos_diagnostico = df_paciente[['COD_DIAGNOSTICO1', 'COD_DIAGNOSTICO2', 'COD_DIAGNOSTICO3', 'COD_DIAGNOSTICO4',
                                       'COD_DIAGNOSTICO5', 'COD_DIAGNOSTICO6', 'COD_DIAGNOSTICO7','COD_DIAGNOSTICO8',
                                       'COD_DIAGNOSTICO9','COD_DIAGNOSTICO10']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def obtener_diagnosticos_ieeh(df, nro_identificacion):
    df_paciente = df[df['RUT'] == nro_identificacion]
    diagnosticos = df_paciente[['DIAG1','DIAG2','DIAG3']].values.flatten()
    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]

    return list(set(diagnosticos))

def obtener_diagnosticos_urgencias(df, nro_identificacion):
    df_paciente = df[df['RUT'] == nro_identificacion]
    diagnosticos = df_paciente[['DESCDIAGNOSTICO']].values.flatten()
    codigos_diagnostico = df_paciente[['CODDIAGNOSTICO']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def obtener_diagnosticos_inter(df, nro_identificacion):
    df_paciente = df[df['NRO_IDENTIFICADOR'] == nro_identificacion]
    diagnosticos = df_paciente[['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']].values.flatten()
    codigos_diagnostico = df_paciente[['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def agregar_diagnosticos_y_codigos(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('NRO_IDENTIFICADOR_PACIENTE').agg({'NRO_IDENTIFICADOR_PACIENTE': 'first', 'DIAGNOSTICO1': 'first',
                                                            'DIAGNOSTICO2': 'first','DIAGNOSTICO3': 'first','DIAGNOSTICO4': 'first',
                                                            'DIAGNOSTICO5': 'first','DIAGNOSTICO6': 'first','DIAGNOSTICO7': 'first',
                                                            'DIAGNOSTICO8': 'first','DIAGNOSTICO9': 'first','DIAGNOSTICO10': 'first',
                                                            'COD_DIAGNOSTICO1': 'first','COD_DIAGNOSTICO2': 'first', 'COD_DIAGNOSTICO2': 'first',
                                                            'COD_DIAGNOSTICO3': 'first', 'COD_DIAGNOSTICO4': 'first', 'COD_DIAGNOSTICO5': 'first',
                                                            'COD_DIAGNOSTICO6': 'first', 'COD_DIAGNOSTICO7': 'first','COD_DIAGNOSTICO8': 'first',
                                                                'COD_DIAGNOSTICO8': 'first',  'COD_DIAGNOSTICO9': 'first',
                                                                'COD_DIAGNOSTICO10': 'first'}).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['NRO_IDENTIFICADOR_PACIENTE'].apply(lambda x: obtener_diagnosticos_y_codigos(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'NRO_IDENTIFICADOR_PACIENTE': 'RUT'})
    return resultados

def agregar_diagnosticos_ieeh(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT', 'DESCDIAGNOSTICO', 'CODDIAGNOSTICO', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('RUT').agg({'RUT': 'first', 'DIAG1': 'first',
                                                            'DIAG1': 'first','DIAG3': 'first'}).reset_index(drop=True)
    #resultados['CODIGOS_DIAGNOSTICO'] = []
    resultados['DIAGNOSTICOS'] = resultados['RUT'].apply(lambda x: obtener_diagnosticos_ieeh(resultado_filtrado, x))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'RUT': 'RUT'})
    resultados['CODIGOS_DIAGNOSTICO'] = np.nan
    return resultados

def agregar_diagnosticos_y_codigos_urgencias(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT', 'DESCDIAGNOSTICO', 'CODDIAGNOSTICO', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    resultados = resultado_filtrado.groupby('RUT').agg({'RUT': 'first', 'DESCDIAGNOSTICO': 'first',
                                                        'CODDIAGNOSTICO': 'first'}).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['RUT'].apply(lambda x: obtener_diagnosticos_urgencias(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'RUT': 'RUT'})

    return resultados

def agregar_diagnosticos_y_codigos_inter(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('NRO_IDENTIFICADOR').agg({'NRO_IDENTIFICADOR': 'first', 'SOSP_DIAG_DESC_1': 'first',
                                                            'SOSP_DIAG_DESC_2': 'first','SOSP_DIAG_DESC_3': 'first','SOSP_DIAG_DESC_4': 'first',
                                                            'SOSP_DIAG_DESC_5': 'first','SOSP_DIAG_COD_1': 'first','SOSP_DIAG_COD_2': 'first',
                                                                      'SOSP_DIAG_COD_3': 'first',
                                                            'SOSP_DIAG_COD_4': 'first', 'SOSP_DIAG_COD_5': 'first'
                                                            }).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['NRO_IDENTIFICADOR'].apply(lambda x: obtener_diagnosticos_inter(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'NRO_IDENTIFICADOR': 'RUT'})
    return resultados

def crear_dataframe_ambulatorio(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cáncer de Próstata',
        'RUT': resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['PACIENTE'],
        
        'N° Telefono 1': resultado_filtrado.get('TELEFONO_MOVIL', np.nan),
        'N° Telefono 2':resultado_filtrado.get('TELEFONO_FIJO_1', np.nan),
        'N° Telefono 3': resultado_filtrado.get('TELEFONO_FIJO_2', np.nan),
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })
   
    return nuevo_dataframe

def crear_dataframe_ieeh(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cáncer de Próstata',
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOMBRES']+ ' '+ resultado_filtrado['APELL_PATE']+ ' '+ resultado_filtrado['APELL_MATE'] ,
        'N° Telefono 1': resultado_filtrado['TELEFONO'],
        'N° Telefono 2': np.nan,
        'N° Telefono 3': np.nan,
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def crear_dataframe_urgencias(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cáncer de Próstata',
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente': resultado_filtrado['NOMBRES']+ ' ' + resultado_filtrado['APELL_PATE']+ ' '+ resultado_filtrado['APELL_MATE'] ,
        'N° Telefono 1': resultado_filtrado['TELEFONO'],
        'N° Telefono 2': np.nan,
        'N° Telefono 3': np.nan,
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def crear_dataframe_urgencias(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cáncer de Próstata',
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOMBRES_PACIENTE'] + ' ' + resultado_filtrado['APELLIDOS_PACIENTE'],
        'N° Telefono 1': np.nan,
        'N° Telefono 2': np.nan,
        'N° Telefono 3': np.nan,
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def crear_dataframe_inter(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cáncer de Próstata',
        'RUT': resultado_filtrado['NRO_IDENTIFICADOR'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOM_PAC'] + ' ' + resultado_filtrado['APELLPAT_PAC']+' '+resultado_filtrado['APELLMAT_PAC'],
        'N° Telefono 1': resultado_filtrado['FONO_MOVIL'],
        'N° Telefono 2': resultado_filtrado['FONO_FIJO_1'],
        'N° Telefono 3': resultado_filtrado['FONO_FIJO_2'],
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def safe_convert_to_int(value):
    try:
        return int(value)
    except ValueError:
        return value

def merge_ambulatorio(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    # Realizar merge por la columna 'RUT'
    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.
    
    return merged_df 

def merge_urgencia(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    # Realizar merge por la columna 'RUT'
    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.

    return merged_df

def merge_inter(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    # Realizar merge por la columna 'RUT'
    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.

    return merged_df

def filtrar_columnas_relevantes(merged_df):

    merged_df['RUT'] = merged_df['RUT'].astype(str)
    merged_df = merged_df[~(merged_df['RUT'].str.contains('A', case=False, na=False))]
    nuevo_dataframe = merged_df.drop_duplicates(subset='RUT', keep='first')
    nuevo_dataframe = nuevo_dataframe[['Problema de Salud','RUT' ,'DV',
       'Nombre del Paciente', 'N° Telefono 1', 'N° Telefono 2',
       'N° Telefono 3', 'Fecha Proximo Hito', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO',
       'Nombre agenda Medico especialista']]
    nuevo_dataframe = nuevo_dataframe.rename(columns={'Problema de Salud': 'Problema Oncologico'})
    nuevo_dataframe['Nombre garantia'] = np.nan
    nuevo_dataframe= nuevo_dataframe[['Problema Oncologico',
    'RUT',
    'DV',
    'Nombre del Paciente',
    'N° Telefono 1',
    'N° Telefono 2',
    'N° Telefono 3',
    'Nombre garantia',
    'Fecha Proximo Hito',
    'DIAGNOSTICOS',
    'CODIGOS_DIAGNOSTICO',
    'Nombre agenda Medico especialista']]
    return nuevo_dataframe

def crear_dataframe_ges(dataframe_tiroides):
    # Crear un nuevo DataFrame con las columnas deseadas
    nuevo_dataframe_2 = pd.DataFrame({
        'Problema Oncologico': dataframe_tiroides['Problema de salud'],
        'RUT': dataframe_tiroides['RUT'],
        'DV': dataframe_tiroides['DV'],
        'Nombre del Paciente':dataframe_tiroides['Nombre'],
        'N° Telefono 1': dataframe_tiroides['Telefonos.x'],
        'N° Telefono 2': dataframe_tiroides['Telefonos.y'],
        'N° Telefono 3': np.nan,
        'Nombre garantia': dataframe_tiroides['Nombre garantia'],
        'Fecha Proximo Hito': np.nan,
        'DIAGNOSTICOS': np.nan,
        'CODIGOS_DIAGNOSTICO': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })
    return nuevo_dataframe_2

def crear_dataframe_ultima_atencion(dataframe_tiroides):
    # Crear un nuevo DataFrame con las columnas deseadas
    nuevo_dataframe_2 = pd.DataFrame({
        'RUT': dataframe_tiroides['RUT'],
        'Fecha Ultima atencion': dataframe_tiroides['HORA_TERMINO_ATENCION'],
        'Profesional Ultima atencion': dataframe_tiroides['ESTAMENTO_PROFESIONAL_ATENCION']+ ' '+
        dataframe_tiroides['NOMBRE_PROFESIONAL_ATENCION']+ ' ' + dataframe_tiroides['DESCRIPCION'] + ' '+
        dataframe_tiroides['ESPECIALIDAD_AGENDA']
    })
    return nuevo_dataframe_2

def proceso_dataframe(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides(dataframe)
        resultado_filtrado = agregar_dv(resultado_filtrado)
        resultados = agregar_diagnosticos_y_codigos(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_ambulatorio(resultado_filtrado)
        merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales 

def proceso_dataframe_ultimo(ruta_archivo):
    dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
    resultado_filtrado = filtro_dataframe_ca_tiroides(dataframe)
    resultado_filtrado = agregar_dv(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_ambulatorio(resultado_filtrado)
    merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
    return nuevo_dataframe

def proceso_dataframe_ieeh(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ieeh(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides_ieeh(dataframe)
        resultados = agregar_diagnosticos_ieeh(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_ieeh(resultado_filtrado)
        merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales

def proceso_dataframe_urgencias(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ieeh(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides_urgencias(dataframe)
        resultado_filtrado = agregar_dv_urgencias(resultado_filtrado)
        resultados = agregar_diagnosticos_y_codigos_urgencias(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_urgencias(resultado_filtrado)
        merged_df = merge_urgencia(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales

def proceso_dataframe_inter(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada'])]
    dataframe = dataframe[~dataframe['NRO_IDENTIFICADOR'].str.contains('AT')]
    # Definir la fecha de inicio para el filtro
    fecha_hoy = datetime.today()
    fecha_inicio = fecha_hoy - timedelta(days=10)

    fecha_inicio = pd.to_datetime(fecha_inicio, format='%d/%m/%Y')
    # Filtrar el DataFrame
    dataframe = dataframe[dataframe['FECHA_SOLIC'] >= fecha_inicio]
    resultado_filtrado = filtro_dataframe_ca_tiroides_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado)
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def ultimo_csv_en_carpeta(ruta):
    # Obtener lista de archivos en la carpeta
    archivos = os.listdir(ruta)
    
    # Filtrar archivos que sean .csv
    archivos_csv = [archivo for archivo in archivos if archivo.endswith('.csv')]
    
    if not archivos_csv:
        return None  # Si no hay archivos .csv en la carpeta
    
    # Ordenar la lista de archivos por fecha de modificación
    archivos_csv.sort(key=lambda x: os.path.getmtime(os.path.join(ruta, x)), reverse=True)
    
    # Devolver la ruta completa del último archivo .csv
    return os.path.join(ruta, archivos_csv[0])

def convertir_fecha(fecha):
    try:
        return pd.to_datetime(fecha, format='%d/%m/%Y %H:%M')
    except ValueError:
        return pd.to_datetime(fecha, format='%d-%m-%y %H:%M')

def obtener_fechas_amb(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
        dataframe['DV'] = dataframe['NRO_IDENTIFICADOR_PACIENTE'].str[-1]
        dataframe['RUT'] = dataframe['NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
        



        resultados_totales.append(dataframe)
        
    dataframe = pd.concat(resultados_totales)

    
    # Filtrar por las especialidades de Endocrinología Adulto y Endocrinología Infantil
    endocrinologia_df = dataframe[dataframe['ESPECIALIDAD_AGENDA'].isin(['Urología Adulto'])]
    endocrinologia_df = endocrinologia_df[endocrinologia_df['ESTAMENTO_PROFESIONAL_ATENCION']=='Médico']
    # Filtrar las filas que no contienen 'No realizada' o 'Anulada' en la columna 'ESTADO_CITA'
    endocrinologia_df = endocrinologia_df[~endocrinologia_df['ESTADO_CITA'].isin(['No realizada', 'Anulada'])]
    # Convertir la columna 'HORA_TERMINO_ATENCION' a formato de fecha y hora
    #endocrinologia_df['HORA_TERMINO_ATENCION'] = pd.to_datetime(endocrinologia_df['HORA_TERMINO_ATENCION'], format='mixed')
    endocrinologia_df['HORA_TERMINO_ATENCION'] = endocrinologia_df['HORA_TERMINO_ATENCION'].apply(convertir_fecha)

    # Agrupar por RUT y obtener la última fecha de la hora de atención para cada grupo
    #ultima_fecha_por_rut = endocrinologia_df.groupby('RUT').agg({'HORA_TERMINO_ATENCION': 'max'}).reset_index()

    ultima_fecha_por_rut =  endocrinologia_df.groupby('RUT').agg({
    'HORA_TERMINO_ATENCION': 'max',
    'NOMBRE_PROFESIONAL_ATENCION': 'first',
    'ESTAMENTO_PROFESIONAL_ATENCION': 'first',
    'DESCRIPCION': 'first',
    'ESPECIALIDAD_AGENDA': 'first'
}).reset_index()
    # Fusionar el DataFrame original con el DataFrame resultante para recuperar NOMBRE_PROFESIONAL_ATENCION y ESTAMENTO_PROFESIONAL_ATENCION
    resultado = ultima_fecha_por_rut.merge(endocrinologia_df[['RUT', 'HORA_TERMINO_ATENCION', 'NOMBRE_PROFESIONAL_ATENCION', 'ESTAMENTO_PROFESIONAL_ATENCION','DESCRIPCION']], on=['RUT', 'HORA_TERMINO_ATENCION'], how='left')

    return ultima_fecha_por_rut 

# Ruta archivos

In [13]:
dir_actual = os.getcwd()
dir_archivo_csv = os.path.join(dir_actual,'..','Reporteria.lnk')

In [14]:
import win32com.client

def get_target_path(lnk_file):
    shell = win32com.client.Dispatch("WScript.Shell")
    shortcut = shell.CreateShortCut(lnk_file)
    return shortcut.Targetpath

# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Reporteria.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_reporteria = get_target_path(ruta_del_lnk)

#print("Ruta de destino:", ruta_reporteria)


In [15]:
# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Unidad_Gestion_Del_Paciente.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_ges = get_target_path(ruta_del_lnk)

#print("Ruta de destino:", ruta_ges)

# Rutas Históricas 

In [16]:

"""
#################### RUTA ARCHIVOS ##########################################

ruta_archivo_4 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', '202401_Reporte_Ambulatorio_Mensual.csv')
ruta_archivo_5 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '20240401_Reporte_Ambulatorio_Diario.csv')


# Lista de rutas de archivos
rutas_archivos = [ruta_archivo_4, ruta_archivo_5]

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)
# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)





#Reportes Ambulatorios

#agregamos los dias 
for i in range(1,int(day_formatted)+1):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_Abril', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
    rutas_archivos.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
for i in range(1,today.month):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Mensual Ambulatorio', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))
    rutas_archivos.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))

for i in range(1,6):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Anual Ambulatorio', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
    rutas_archivos.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Anual', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
  

#Reportes IEEH

rutas_archivos_2 = []
for i in range(1,int(day_formatted)+1):
    numero_formateado = f'{i:02}'
    #rutas_archivos_2.append(os.path.join(dir_datos_hlf, 'BDD_Abril', f'{numero_formateado}-{month_formatted}-{today.year}-Informe_Diario-IEEE.csv'))
    rutas_archivos_2.append(os.path.join(ruta_reporteria,'Reporte_IEEH','Diario', f'{numero_formateado}-{month_formatted}-{today.year}-Informe_Diario-IEEE.csv'))

for i in range(1,today.month):
    numero_formateado = f'{i:02}'    
    #rutas_archivos_2.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Mensual IEEH', f'{numero_formateado}-{today.year}-Informe_Mensual-IEEE.csv'))
    rutas_archivos_2.append(os.path.join(ruta_reporteria,'Reporte_IEEH','Mensual', f'{numero_formateado}-{today.year}-Informe_Mensual-IEEE.csv'))

for i in range(1,13):
    numero_formateado = f'{i:02}'
    #rutas_archivos_2.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Mensual IEEH', f'{numero_formateado}-{today.year-1}-Informe_Mensual-IEEE.csv'))
    rutas_archivos_2.append(os.path.join(ruta_reporteria,'Reporte_IEEH','Mensual', f'{numero_formateado}-{today.year-1}-Informe_Mensual-IEEE.csv'))

for i in range(1,5):
    numero_formateado = f'{i:02}'
    #rutas_archivos_2.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Anual IEEH', f'{today.year-i-1}-Informe_Anual-IEEE.csv'))
    rutas_archivos_2.append(os.path.join(ruta_reporteria,'Reporte_IEEH','Anual', f'{today.year-i-1}-Informe_Anual-IEEE.csv'))

#Reportes Urgencias
os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Diario')
rutas_archivos_3 = []
for i in range(1,int(day_formatted)+1):
    numero_formateado = f'{i:02}'
    #rutas_archivos_3.append(os.path.join(dir_datos_hlf, 'BDD_Abril', f'{numero_formateado}-{month_formatted}-{today.year}-Urgencias_atenciones.csv'))
    rutas_archivos_3.append(os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Diario', f'{numero_formateado}-{month_formatted}-{today.year}-Urgencias_atenciones.csv'))

for i in range(1,today.month):
    numero_formateado = f'{i:02}'
    #rutas_archivos_3.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Mensual Urgencias', f'{numero_formateado}-{today.year}-urgencias_atenciones-mensual.csv'))
    rutas_archivos_3.append(os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Mensual', f'{numero_formateado}-{today.year}-urgencias_atenciones-mensual.csv'))

for i in range(1,6):
    numero_formateado = f'{i:02}'
    #rutas_archivos_3.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Anual Urgencias', f'{today.year-i}-Urgencias_atenciones_Anual.csv'))
    rutas_archivos_3.append(os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Anual', f'{today.year-i}-Urgencias_atenciones_Anual.csv'))

"""

# Término de Rutas Históricas

In [17]:
patron = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '*Reporte_Ambulatorio_Diario*')

archivos = glob.glob(patron)

archivos_ordenados = sorted(archivos, key=os.path.getmtime, reverse = True)

ultimos_10_archivos = archivos_ordenados[:10]

for archivo in ultimos_10_archivos:
    fecha_modificacion = datetime.fromtimestamp(os.path.getmtime(archivo))
    #print(f"{archivo}- modificación: {fecha_modificacion}")

In [18]:


#################### RUTA ARCHIVOS ##########################################

dias_reportes = 10

# Lista de rutas de archivos
rutas_archivos = []

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)
# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)


#Reportes Ambulatorios Histórico
ruta_archivo_4 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', '202401_Reporte_Ambulatorio_Mensual.csv')
ruta_archivo_5 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '20240401_Reporte_Ambulatorio_Diario.csv')


# Lista de rutas de archivos
rutas_archivos_hist = [ruta_archivo_4, ruta_archivo_5]
#agregamos los dias 
for i in range(1,int(day_formatted)+1):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_Abril', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
for i in range(1,today.month):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Mensual Ambulatorio', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))

for i in range(1,6):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Anual Ambulatorio', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Anual', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
  



#Reportes Ambulatorios


patron_amb = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '*Reporte_Ambulatorio_Diario*')

archivos_amb = glob.glob(patron_amb)

archivos_ordenados_amb = sorted(archivos_amb, key=os.path.getmtime, reverse = True)

rutas_archivos = archivos_ordenados_amb[:dias_reportes]



#Reportes IEEH

rutas_archivos_2 = []


patron_ieeh = os.path.join(ruta_reporteria,'Reporte_IEEH','Diario', '*Informe_Diario-IEEE*')

archivos_ieeh  = glob.glob(patron_ieeh )

archivos_ordenados_ieeh  = sorted(archivos_ieeh , key=os.path.getmtime, reverse = True)

rutas_archivos_2 = archivos_ordenados_ieeh[:dias_reportes]


#Reportes Urgencias


rutas_archivos_3 = []


patron_urg = os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Diario', '*Urgencias_atenciones*')

archivos_urg  = glob.glob(patron_urg)

archivos_ordenados_urg  = sorted(archivos_urg , key=os.path.getmtime, reverse = True)

rutas_archivos_3 = archivos_ordenados_urg[:dias_reportes]




"\n#Reportes Ambulatorios\n\n\npatron_amb = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '*Reporte_Ambulatorio_Diario*')\n\narchivos_amb = glob.glob(patron_amb)\n\narchivos_ordenados_amb = sorted(archivos_amb, key=os.path.getmtime, reverse = True)\n\nrutas_archivos = archivos_ordenados_amb[:dias_reportes]\n\n\n\n#Reportes IEEH\n\nrutas_archivos_2 = []\n\n\npatron_ieeh = os.path.join(ruta_reporteria,'Reporte_IEEH','Diario', '*Informe_Diario-IEEE*')\n\narchivos_ieeh  = glob.glob(patron_ieeh )\n\narchivos_ordenados_ieeh  = sorted(archivos_ieeh , key=os.path.getmtime, reverse = True)\n\nrutas_archivos_2 = archivos_ordenados_ieeh[:dias_reportes]\n\n\n#Reportes Urgencias\n\n\nrutas_archivos_3 = []\n\n\npatron_urg = os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Diario', '*Urgencias_atenciones*')\n\narchivos_urg  = glob.glob(patron_urg)\n\narchivos_ordenados_urg  = sorted(archivos_urg , key=os.path.getmtime, reverse = True)\n\nrutas_archivos_3 = archivos_ordenad

In [19]:
#Reportes SIC

patron_sic = os.path.join(ruta_reporteria,'Reportes_Solicitudes','Mensual', '*Informe_Solicitudes_DerivacionCE*')

archivos_sic  = glob.glob(patron_sic)

archivos_ordenados_sic  = sorted(archivos_sic , key=os.path.getmtime, reverse = True)

rutas_archivos_4 = archivos_ordenados_sic[:1]


# Generar dataframes

In [20]:
# Llamar a la función proceso_dataframe con la lista de rutas de archivos
resultado = obtener_fechas_amb(rutas_archivos_hist)

#resultado = obtener_fechas_amb(rutas_archivos_hist)

resultado_final = proceso_dataframe(rutas_archivos)

resultado_final_2 = proceso_dataframe_ieeh(rutas_archivos_2)

resultado_final_3 = proceso_dataframe_urgencias(rutas_archivos_3)

resultado_final_4 = proceso_dataframe_inter(rutas_archivos_4[0])

In [21]:



#os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')
ruta_del_archivo = os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')


# Carga el archivo XLSX en un DataFrame
dataframe = pd.read_excel(ruta_del_archivo)

In [22]:
# Carga el archivo XLSX en un DataFrame
dataframe = pd.read_excel(ruta_del_archivo)

# Construir la ruta completa al archivo CSV
dataframe['RUT']= dataframe['RUT'].apply(lambda x : str(x))
dataframe['RUT COMPLETO'] = dataframe['RUT'].astype(str)  + dataframe['DV'].astype(str)
dataframe_tiroides = dataframe[dataframe['Problema de salud'].isin(['Cáncer de Próstata . {decreto nº 228}'])]

dataframe_tiroides["Fecha inicio"]=pd.to_datetime(dataframe_tiroides["Fecha inicio"])
# Ordenar en orden descendente por la columna "FECHA_CITA"
dataframe_tiroides = dataframe_tiroides.sort_values(by='Fecha inicio', ascending=False)


# Eliminar duplicados basados en la columna "NRO_IDENTIFICADOR_PACIENTE"
dataframe_tiroides = dataframe_tiroides.drop_duplicates(subset='RUT', keep='first')

nuevo_dataframe_2 = crear_dataframe_ges(dataframe_tiroides)

resultado_final.append(nuevo_dataframe_2)



#if ultimo_csv:
#    print("La dirección del último archivo .csv en la carpeta es:", ultimo_csv)
    
#    resultado_final.append(proceso_dataframe_ultimo(ultimo_csv))
#    df_concatenated = pd.concat(resultado_final)

#else:
#    print("No se encontraron archivos .csv en la carpeta.")
#    df_concatenated = pd.concat(resultado_final)
 # Concatenar los DataFrames
resultado_final.extend(resultado_final_2)
resultado_final.extend(resultado_final_3)
resultado_final.extend([resultado_final_4])
df_concatenated = pd.concat(resultado_final)






# Eliminar duplicados basados en la columna "NRO_IDENTIFICADOR_PACIENTE"

#consolidated_df = df_concatenated.groupby('RUT').apply(lambda x: x.fillna(method='ffill').fillna(method='bfill')).drop_duplicates().reset_index(drop=True)

#resultado_filtrado_2 = consolidated_df.drop_duplicates(subset='RUT', keep='first')

resultado_filtrado_2 = df_concatenated.drop_duplicates(subset='RUT', keep='first')

# Iterar sobre las filas del primer DataFrame
for index, row in resultado_filtrado_2.iterrows():
    rut = row['RUT']
    
    # Buscar el valor correspondiente en el segundo DataFrame
    nombre_garantia = nuevo_dataframe_2.loc[nuevo_dataframe_2['RUT'] == str(rut), 'Nombre garantia'].values
    
    # Encontrar la fila correspondiente en el primer DataFrame y asignar el valor
    resultado_filtrado_2.loc[resultado_filtrado_2['RUT'] == rut, 'Nombre garantia'] = nombre_garantia[0] if nombre_garantia.size > 0 else None

resultado_filtrado_2['RUT'] = resultado_filtrado_2['RUT'].apply(lambda x: str(x))


In [23]:
#nombre_archivo = 'Total pacientes_nuevos_ca_tiroide.xlsx'  # Nombre del archivo de Excel
#resultado_filtrado_2.to_excel(nombre_archivo, index=False)  # index=False para no incluir el índice en el archivo de Excel


In [24]:
resultado_filtrado_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
3043,Cáncer de Próstata . {decreto nº 228},6120935,2,"NEIRA OLGUÍN, HERNÁN DEL CARMEN",- / - / / 77017288 / 77017288 / - / -/.,#ND,NaN,Etapificación,NaN,NaN,NaN,NaN
2469,Cáncer de Próstata . {decreto nº 228},5606495,8,"GUTIÉRREZ DONOSO, PABLO OMAR",#ND,#ND,NaN,Etapificación,NaN,NaN,NaN,NaN
4569,Cáncer de Próstata . {decreto nº 228},7847155,7,"PIZARRO LARA, PEDRO GALVARINO",#ND,#ND,NaN,Etapificación,NaN,NaN,NaN,NaN
923,Cáncer de Próstata . {decreto nº 228},3680199,9,"MARTÍNEZ MUÑOZ, HUGO RENÉ",- / - / 0 / / - / 22891060 / -//22891060/.,#ND,NaN,Etapificación,NaN,NaN,NaN,NaN
3933,Cáncer de Próstata . {decreto nº 228},7055450,K,"HERNÁNDEZ RODRÍGUEZ, MARCELINO ENRIQUE",- / - / 22884774 / 22884774 / 76966849 / 22884...,#ND,NaN,Etapificación,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14,Cáncer de Próstata,49957845,8,JOSE ANTONIO RODRIGUEZ RUIZ,NaN,NaN,NaN,None,NaN,[CANCER DE PROSTATA],[nan],NaN
0,Cáncer de Próstata,4093194,5,Juan Meneses Meneses,21965612,36602682,NaN,None,NaN,"[Tumor maligno de la próstata, ( de los órgano...","[nan, C61]",NaN
1,Cáncer de Próstata,5939305,7,Osvaldo Del Rosario Savignones Muñoz,66681637,2857500,NaN,None,NaN,"[Tumor maligno de la próstata, ( de los órgano...","[nan, C61]",NaN
2,Cáncer de Próstata,5974881,5,Jaime Nelson Rey Martel,40594881,86694344,NaN,None,NaN,"[Tumor maligno de la próstata, ( de los órgano...","[nan, C61]",NaN


In [25]:
resultado_filtrado_2 = resultado_filtrado_2.drop_duplicates(subset='RUT', keep='first')

In [26]:
resultado_filtrado_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
3043,Cáncer de Próstata . {decreto nº 228},6120935,2,"NEIRA OLGUÍN, HERNÁN DEL CARMEN",- / - / / 77017288 / 77017288 / - / -/.,#ND,NaN,Etapificación,NaN,NaN,NaN,NaN
2469,Cáncer de Próstata . {decreto nº 228},5606495,8,"GUTIÉRREZ DONOSO, PABLO OMAR",#ND,#ND,NaN,Etapificación,NaN,NaN,NaN,NaN
4569,Cáncer de Próstata . {decreto nº 228},7847155,7,"PIZARRO LARA, PEDRO GALVARINO",#ND,#ND,NaN,Etapificación,NaN,NaN,NaN,NaN
923,Cáncer de Próstata . {decreto nº 228},3680199,9,"MARTÍNEZ MUÑOZ, HUGO RENÉ",- / - / 0 / / - / 22891060 / -//22891060/.,#ND,NaN,Etapificación,NaN,NaN,NaN,NaN
3933,Cáncer de Próstata . {decreto nº 228},7055450,K,"HERNÁNDEZ RODRÍGUEZ, MARCELINO ENRIQUE",- / - / 22884774 / 22884774 / 76966849 / 22884...,#ND,NaN,Etapificación,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14,Cáncer de Próstata,49957845,8,JOSE ANTONIO RODRIGUEZ RUIZ,NaN,NaN,NaN,None,NaN,[CANCER DE PROSTATA],[nan],NaN
0,Cáncer de Próstata,4093194,5,Juan Meneses Meneses,21965612,36602682,NaN,None,NaN,"[Tumor maligno de la próstata, ( de los órgano...","[nan, C61]",NaN
1,Cáncer de Próstata,5939305,7,Osvaldo Del Rosario Savignones Muñoz,66681637,2857500,NaN,None,NaN,"[Tumor maligno de la próstata, ( de los órgano...","[nan, C61]",NaN
2,Cáncer de Próstata,5974881,5,Jaime Nelson Rey Martel,40594881,86694344,NaN,None,NaN,"[Tumor maligno de la próstata, ( de los órgano...","[nan, C61]",NaN


In [27]:
resultado_filtrado_2 = resultado_filtrado_2.drop_duplicates(subset='RUT', keep='first')

# Cruce BDD Gestoras

In [28]:
dir_actual = os.getcwd()
os.listdir(os.path.join(dir_actual, '..','CA UROLOGICOS'))

['20240711_citas_futuras_renal.xlsx',
 '20240711_citas_futuras_testicular.xlsx',
 '20240711_citas_futuras_vesical.xlsx',
 '20240711_ultima_atencion_renal.xlsx',
 '20240711_ultima_atencion_testicular.xlsx',
 '20240711_ultima_atencion_vesical.xlsx',
 'BASE TRABAJO GESTION ONCOLOGICA_CANCER UROLOGICOS.xlsx',
 'BASE TRABAJO GESTION ONCOLOGICA_CANCER UROLOGICOS_v2.xlsx',
 'Reporte diario casos nuevos']

In [29]:
archivo_excel = os.path.join(dir_actual, '..','CA UROLOGICOS', 'BASE TRABAJO GESTION ONCOLOGICA_CANCER UROLOGICOS.xlsx') # Gestion_Oncologica -> apolo

dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')


In [30]:
dataframe_2

,Problema Oncologico,Rut,DV,Nombre,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cancer de Pulmón,5513331,K,MARÍA MAGALY MIRANDA VÁSQUEZ,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor maligno de los bronquios y del pulmon'],930854428/NO ESPECIFICADO/-,NaN,NaN,24-06-2024,Médico JOSE RAMON SILVA FIGUEROA Derivacion de...


In [31]:
dataframe_2

,Problema Oncologico,Rut,DV,Nombre,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cancer de Pulmón,5513331,K,MARÍA MAGALY MIRANDA VÁSQUEZ,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor maligno de los bronquios y del pulmon'],930854428/NO ESPECIFICADO/-,NaN,NaN,24-06-2024,Médico JOSE RAMON SILVA FIGUEROA Derivacion de...


In [32]:

dataframe = resultado_filtrado_2.copy()

# Obtener la ruta del directorio actual del script
#dir_actual = os.path.dirname(__file__)
#dir_actual = os.getcwd()

dir_actual = os.getcwd()
os.listdir(os.path.join(dir_actual, '..','CA UROLOGICOS'))

# Construir la ruta completa al archivo CSV
archivo_excel = os.path.join(dir_actual, '..','CA UROLOGICOS', 'BASE TRABAJO GESTION ONCOLOGICA_CANCER UROLOGICOS.xlsx') # Gestion_Oncologica -> apolo

dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')


# Obtener la cantidad de filas que no son nulas en la columna 'mi_columna'
cantidad_filas_no_nulas = dataframe_2['Rut'].count()



print("Cantidad de pacientes en la base", cantidad_filas_no_nulas)



dataframe_2 = dataframe_2[:cantidad_filas_no_nulas]

dataframe_2.reset_index(drop=True, inplace=True)


# Renombrar la columna 'old_name' a 'new_name'
dataframe_2 = dataframe_2.rename(columns={'Rut': 'RUT'})


dataframe_2 = dataframe_2.rename(columns={'Problema de Salud': 'Problema Oncologico'})

dataframe_2 = dataframe_2.rename(columns={'Nombre': 'Nombre del Paciente'})

# Definir una función para quitar los últimos dos dígitos
def quitar_ultimos_dos(elemento):
    return np.int64(elemento[:-2])

# Convertir todos los elementos de la columna 'RUT' a string
dataframe_2['RUT'] = dataframe_2['RUT'].astype(str)

# Aplicar la función a la columna 'RUT'
#dataframe_2['RUT'] = dataframe_2['RUT'].apply(quitar_ultimos_dos)



# Identificar las filas donde 'DV' es NaN
filas_dv_nan = dataframe_2[dataframe_2['DV'].isna()]

# Iterar sobre estas filas y corregir los valores de 'DV' y 'Rut'
for index, fila in filas_dv_nan.iterrows():
    rut = str(fila['RUT'])
    if True:
      dv = rut[-1]  # Último dígito del Rut
      dataframe_2.at[index, 'DV'] = dv
      dataframe_2.at[index, 'RUT'] = rut[:-1]  # Eliminar el último dígito del Rut


dataframe_2 = dataframe_2[~(dataframe_2['RUT'].str.contains('A', case=False, na=False))]

dataframe_2['RUT'] = dataframe_2['RUT'].astype(float)
dataframe_2['RUT'] = dataframe_2['RUT'].astype(int)


Cantidad de pacientes en la base 1


In [33]:
list(dataframe_2.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Etapa',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Fecha de Comité',
 'Observacion',
 'Diagnosticos',
 'Telefonos',
 'CITAS FUTURAS',
 'MEDICO',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']

In [34]:
dataframe_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cancer de Pulmón,5513331,K,MARÍA MAGALY MIRANDA VÁSQUEZ,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor maligno de los bronquios y del pulmon'],930854428/NO ESPECIFICADO/-,NaN,NaN,24-06-2024,Médico JOSE RAMON SILVA FIGUEROA Derivacion de...


In [35]:
list(dataframe_2.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Etapa',
 'Estado',
 'Fecha Próximo Hito',
 'Medico Responsable',
 'Fecha de Comité',
 'Observacion',
 'Diagnosticos',
 'Telefonos',
 'CITAS FUTURAS',
 'MEDICO',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']

# Merge Total pacientes vs Gestoras

In [36]:
# Ejemplo de creación de DataFrame (puedes cargar tus propios DataFrames)
df1 = dataframe.copy()
df2 = dataframe_2.copy()

# Agrega una columna para identificar el origen de cada RUT
df1['_df'] = 'df1'
df2['_df'] = 'df2'

# Convertir la columna 'RUT' en ambos DataFrames al mismo tipo de datos
df1['RUT'] = df1['RUT'].astype(str)
df2['RUT'] = df2['RUT'].astype(str)

#print(f"df_1: {df1['RUT'] }")
#print(f"df_2: {df2['RUT'] }")

# Realiza un merge de los dos DataFrames en la columna 'RUT' y conserva solo las filas que aparecen en 'left_only'
df_merge = pd.merge(df1, df2, on='RUT', how='outer', indicator=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['Nombre del Paciente_x'].fillna(df_merge['Nombre del Paciente_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Nombre del Paciente_x': 'Nombre del Paciente'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['Nombre del Paciente_y'], inplace=True)

# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['Problema Oncologico_x'].fillna(df_merge['Problema Oncologico_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Problema Oncologico_x': 'Problema Oncologico'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['Problema Oncologico_y'], inplace=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['DV_x'].fillna(df_merge['DV_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'DV_x': 'DV'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['DV_y'], inplace=True)

df_ca_tiroide = df_merge[df_merge['_merge']=='left_only']

# Reemplazar los valores NaN por '-'
df_ca_tiroide['N° Telefono 1'] = df_ca_tiroide['N° Telefono 1'].fillna('-')
df_ca_tiroide['N° Telefono 2'] = df_ca_tiroide['N° Telefono 2'].fillna('-')
df_ca_tiroide['N° Telefono 3'] = df_ca_tiroide['N° Telefono 3'].fillna('-')

def combinar_telefonos(row):
    return '/'.join(filter(None, [str(row['N° Telefono 1']), str(row['N° Telefono 2']), str(row['N° Telefono 3'])]))

# Aplicar la función a lo largo de las filas del DataFrame para crear la columna 'Telefono Consolidado'
df_ca_tiroide_telefono = df_ca_tiroide[['N° Telefono 1','N° Telefono 2','N° Telefono 3']]
df_ca_tiroide['Telefono Consolidado2'] = df_ca_tiroide_telefono.apply(combinar_telefonos, axis=1)

# Eliminar las columnas originales de los números de teléfono si ya no son necesarias
df_ca_tiroide.drop(columns=['N° Telefono 1', 'N° Telefono 2', 'N° Telefono 3'], inplace=True)

df_ca_tiroide['Etapa'] = np.nan

df_ca_tiroide['Estado'] = np.nan

df_ca_tiroide['Fecha Proximo Hito'] = np.nan

df_ca_tiroide['Observacion'] = np.nan

df_ca_tiroide['Medico Responsable'] = np.nan

df_ca_tiroide['Fecha Ultima atencion'] = np.nan

df_ca_tiroide['Profesional Ultima atencion'] = np.nan

df_ca_tiroide['MEDICO'] = np.nan

df_ca_tiroide['CITAS FUTURAS'] = np.nan

#df_ca_tiroide = df_ca_tiroide[['Problema Oncologico', 'RUT', 'DV', 'Nombre del Paciente','Etapa','Nombre garantia','Estado','Fecha Proximo Hito','Observacion','Medico Responsable','Fecha Ultima atencion', 'Profesional Ultima atencion','DIAGNOSTICOS', 'Telefono Consolidado2']]
df_ca_tiroide = df_ca_tiroide[['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Etapa',
 'Estado',
 'Fecha Próximo Hito',
 'Observacion',
 'Medico Responsable',
 'DIAGNOSTICOS',
 'Telefono Consolidado2',
 'CITAS FUTURAS',
 'MEDICO',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']]

In [37]:
df_ca_tiroide

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Observacion,Medico Responsable,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cáncer de Próstata,10467223,K,NAIM ANTONIO SALVADOR CHOMALÍ,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de la prostata],-/-/-,NaN,NaN,NaN,NaN
1,Cáncer de Próstata,10754884,K,MIGUEL ANGEL ORTIZ SOLORZA,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de la prostata, nan]",///-/-,NaN,NaN,NaN,NaN
2,Cáncer de Próstata,10961756,3,WILSON MIGUEL VERDUGO CERDA,NaN,NaN,NaN,NaN,NaN,[CÁNCER DE PRÓSTATA ETAPA IV],-/-/-,NaN,NaN,NaN,NaN
3,Cáncer de Próstata,11654329,K,PATRICIO ENRIQUE CABRERA ALVARADO,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA./MTT ESPINAL],-/-/-,NaN,NaN,NaN,NaN
4,Cáncer de Próstata . {decreto nº 228},21903974,3,"CUEBA MARTEL, FELIPE",NaN,NaN,NaN,NaN,NaN,NaN,- / - / 22820046 / / - / 22820046 / -/./#ND/-,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Cáncer de Próstata,9233670,0,JORGE ARTURO PEÑA RIFFO,NaN,NaN,NaN,NaN,NaN,[CANCER DE PROSTATA],-/-/-,NaN,NaN,NaN,NaN
109,Cáncer de Próstata,9257543,8,JUAN IGNACIO VÁSQUEZ LEIVA,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA],-/-/-,NaN,NaN,NaN,NaN
110,Cáncer de Próstata . {decreto nº 228},9270788,1,"BRAVIN HERRERA, GIORGIO ALDO",NaN,NaN,NaN,NaN,NaN,NaN,- / - / 26131454 / 0 / 57602865 / 22553514 / -...,NaN,NaN,NaN,NaN
111,Cáncer de Próstata,9384687,7,RODOLFO BRUNO PEDRAZA CORROTEA,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA ],-/-/-,NaN,NaN,NaN,NaN


In [38]:
df_ca_tiroide['Fecha de Comité'] = np.nan

In [39]:
lista = ['Problema Oncologico', 'RUT','DV','Nombre del Paciente','Etapa','Estado','Fecha Próximo Hito','Medico Responsable','Fecha de Comité','Observacion','DIAGNOSTICOS','Telefono Consolidado2','CITAS FUTURAS','MEDICO','Fecha Ultima atencion','Profesional Ultima atencion']

In [40]:
print(f'Cantidad de pacientes nuevos encontrados: {df_ca_tiroide.shape[0]}')

Cantidad de pacientes nuevos encontrados: 112


# Guardar estadística de llegada


In [41]:
fecha_hoy = date.today()
data = {'Problema Oncologico': ['Cáncer de Próstata'], 'Fecha Medición': [fecha_hoy], 'Cantidad de llegada': [df_ca_tiroide.shape[0]], 'Cantidad de pacientes base': [cantidad_filas_no_nulas]}
estadisticas_llegadas = pd.DataFrame(data)

nombre_archivo = f'Estadisticas_llegadas.xlsx'
archivo_excel = os.path.join(dir_actual, '..','Power BI',f'{nombre_archivo}')

df_existente = pd.read_excel(archivo_excel)
df_existente['Fecha Medición'] = pd.to_datetime(df_existente['Fecha Medición'])


wb = Workbook()
ws = wb.active
df_final = pd.concat([df_existente,estadisticas_llegadas], ignore_index=True)


for r in dataframe_to_rows(df_final, index= False, header = True):
    ws.append(r)

for cell in ws['B'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'Estadisticas_llegadas.xlsx'
wb.save(os.path.join(dir_actual, '..','Power BI',f'{nombre_archivo}'))


# Agregar Fallecidos

In [42]:
dir_actual = os.getcwd()
dir_gestoras = os.path.join(dir_actual,'..')

In [43]:
ruta_del_archivo_2 = os.path.join(dir_gestoras, 'FALLECIDOS HLF CORTE 30_04_2024.xlsx')


In [44]:
# Carga el archivo XLSX en un DataFrame
fallecidos = pd.read_excel(ruta_del_archivo_2)

In [45]:
fallecidos_rut = pd.DataFrame({'RUT': fallecidos['RUD']})

In [46]:
import pandas as pd

# Suponiendo que tus dataframes se llaman df_consolidado y df_fallecidos


# Combinamos los dataframes en función de la columna 'RUT'


fallecidos_rut['RUT'] = fallecidos_rut['RUT'].astype(str)  # Convertir la columna a tipo string
fallecidos_rut['RUT'] = fallecidos_rut['RUT'].replace(['-', '\.'], '', regex=True) 

fallecidos_rut['RUT'] = fallecidos_rut['RUT'].astype(str)


In [47]:
df_pacientes_nuevos = df_ca_tiroide[lista].copy()

In [48]:
df_pacientes_nuevos


,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cáncer de Próstata,10467223,K,NAIM ANTONIO SALVADOR CHOMALÍ,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de la prostata],-/-/-,NaN,NaN,NaN,NaN
1,Cáncer de Próstata,10754884,K,MIGUEL ANGEL ORTIZ SOLORZA,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de la prostata, nan]",///-/-,NaN,NaN,NaN,NaN
2,Cáncer de Próstata,10961756,3,WILSON MIGUEL VERDUGO CERDA,NaN,NaN,NaN,NaN,NaN,NaN,[CÁNCER DE PRÓSTATA ETAPA IV],-/-/-,NaN,NaN,NaN,NaN
3,Cáncer de Próstata,11654329,K,PATRICIO ENRIQUE CABRERA ALVARADO,NaN,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA./MTT ESPINAL],-/-/-,NaN,NaN,NaN,NaN
4,Cáncer de Próstata . {decreto nº 228},21903974,3,"CUEBA MARTEL, FELIPE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 22820046 / / - / 22820046 / -/./#ND/-,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Cáncer de Próstata,9233670,0,JORGE ARTURO PEÑA RIFFO,NaN,NaN,NaN,NaN,NaN,NaN,[CANCER DE PROSTATA],-/-/-,NaN,NaN,NaN,NaN
109,Cáncer de Próstata,9257543,8,JUAN IGNACIO VÁSQUEZ LEIVA,NaN,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA],-/-/-,NaN,NaN,NaN,NaN
110,Cáncer de Próstata . {decreto nº 228},9270788,1,"BRAVIN HERRERA, GIORGIO ALDO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 26131454 / 0 / 57602865 / 22553514 / -...,NaN,NaN,NaN,NaN
111,Cáncer de Próstata,9384687,7,RODOLFO BRUNO PEDRAZA CORROTEA,NaN,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA ],-/-/-,NaN,NaN,NaN,NaN


In [49]:
df_pacientes_nuevos.loc[df_pacientes_nuevos['RUT'].isin(fallecidos_rut['RUT']), 'Estado'] = 'Fallecido'


In [50]:
df_pacientes_nuevos.loc[df_pacientes_nuevos['RUT'].isin(fallecidos_rut['RUT']), 'Etapa'] = 'Alta'


In [51]:
df_pacientes_nuevos

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cáncer de Próstata,10467223,K,NAIM ANTONIO SALVADOR CHOMALÍ,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de la prostata],-/-/-,NaN,NaN,NaN,NaN
1,Cáncer de Próstata,10754884,K,MIGUEL ANGEL ORTIZ SOLORZA,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de la prostata, nan]",///-/-,NaN,NaN,NaN,NaN
2,Cáncer de Próstata,10961756,3,WILSON MIGUEL VERDUGO CERDA,NaN,NaN,NaN,NaN,NaN,NaN,[CÁNCER DE PRÓSTATA ETAPA IV],-/-/-,NaN,NaN,NaN,NaN
3,Cáncer de Próstata,11654329,K,PATRICIO ENRIQUE CABRERA ALVARADO,NaN,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA./MTT ESPINAL],-/-/-,NaN,NaN,NaN,NaN
4,Cáncer de Próstata . {decreto nº 228},21903974,3,"CUEBA MARTEL, FELIPE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 22820046 / / - / 22820046 / -/./#ND/-,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Cáncer de Próstata,9233670,0,JORGE ARTURO PEÑA RIFFO,NaN,NaN,NaN,NaN,NaN,NaN,[CANCER DE PROSTATA],-/-/-,NaN,NaN,NaN,NaN
109,Cáncer de Próstata,9257543,8,JUAN IGNACIO VÁSQUEZ LEIVA,NaN,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA],-/-/-,NaN,NaN,NaN,NaN
110,Cáncer de Próstata . {decreto nº 228},9270788,1,"BRAVIN HERRERA, GIORGIO ALDO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 26131454 / 0 / 57602865 / 22553514 / -...,NaN,NaN,NaN,NaN
111,Cáncer de Próstata,9384687,7,RODOLFO BRUNO PEDRAZA CORROTEA,Alta,Fallecido,NaN,NaN,NaN,NaN,[CA PROSTATA ],-/-/-,NaN,NaN,NaN,NaN


In [52]:
df_pacientes_nuevos[df_pacientes_nuevos['Etapa']=='Alta']

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
5,Cáncer de Próstata,2614835,9,BIENVENIDO DEL CARMEN MELLA RUIZ,Alta,Fallecido,NaN,NaN,NaN,NaN,[CA PROSTATA],-/-/-,NaN,NaN,NaN,NaN
7,Cáncer de Próstata,2775956,4,JOSÉ EUGENIO ANSALDO ROMERO,Alta,Fallecido,NaN,NaN,NaN,NaN,[CA PROSTATA],-/-/-,NaN,NaN,NaN,NaN
10,Cáncer de Próstata,2974643,5,ARTURO JULIO GUZMÁN ACHIARDI,Alta,Fallecido,NaN,NaN,NaN,NaN,[CA PROSTATA EN PALIATIVOS],-/-/-,NaN,NaN,NaN,NaN
11,Cáncer de Próstata,3129648,K,JOSÉ MIGUEL SOTO VERA,Alta,Fallecido,NaN,NaN,NaN,NaN,[Tumor maligno de la prostata],-/-/-,NaN,NaN,NaN,NaN
13,Cáncer de Próstata,3359996,K,HUGO O HIGGINS GUERRERO,Alta,Fallecido,NaN,NaN,NaN,NaN,[CANCER PROSTATA],-/-/-,NaN,NaN,NaN,NaN
16,Cáncer de Próstata,3627614,2,JOSÉ LUIS PARRA SEGOVIA,Alta,Fallecido,NaN,NaN,NaN,NaN,[CANCER DE PROSTATA CON METASTASIS OSEAS],-/-/-,NaN,NaN,NaN,NaN
21,Cáncer de Próstata,3980670,3,SEGUNDO DÍAZ CASTRO,Alta,Fallecido,NaN,NaN,NaN,NaN,[Tumor maligno de la prostata],-/-/-,NaN,NaN,NaN,NaN
31,Cáncer de Próstata,4464946,2,MIGUEL GERMÁN REYES VIDELA,Alta,Fallecido,NaN,NaN,NaN,NaN,[CA PROSTATA],-/-/-,NaN,NaN,NaN,NaN
44,Cáncer de Próstata,5191796,0,CARLOS EMETERIO MARTÍNEZ MARDONES,Alta,Fallecido,NaN,NaN,NaN,NaN,"[CA PROSTATA, nan]",02953759201///-/-,NaN,NaN,NaN,NaN
45,Cáncer de Próstata,5200424,1,ISMAEL SEGUNDO JARAMILLO QUINTEROS,Alta,Fallecido,NaN,NaN,NaN,NaN,[CRECIMIENTO PROSTÁTICO --> BENIGNO VS CANCER ...,-/-/-,NaN,NaN,NaN,NaN


In [53]:
df_pacientes_nuevos = df_pacientes_nuevos[~(df_pacientes_nuevos['RUT']=='nan')]

# Guardar dataframe

In [54]:
dir_actual = os.getcwd()
os.listdir(os.path.join(dir_actual, '..','CA UROLOGICOS','Reporte diario casos nuevos'))

['20240515_Pacientes_nuevos_ca_prostata.xlsx',
 '20240515_Pacientes_nuevos_ca_renal.xlsx',
 '20240515_Pacientes_nuevos_ca_testicular.xlsx',
 '20240515_Pacientes_nuevos_ca_vesical.xlsx',
 '20240711_Pacientes_nuevos_ca_renal.xlsx',
 '20240711_Pacientes_nuevos_ca_testicular.xlsx',
 '20240711_Pacientes_nuevos_ca_vesical.xlsx']

In [55]:
# Guardar el DataFrame en un archivo de Excel

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)


nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_Pacientes_nuevos_ca_prostata.xlsx'  # Nombre del archivo de Excel
df_pacientes_nuevos.to_excel(os.path.join(dir_actual, '..','CA UROLOGICOS','Reporte diario casos nuevos',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 

# Convertir la columna 'RUT' en DataFrame_2 a tipo object si es necesario


df_concatenated_2 = pd.concat([dataframe_2,df_pacientes_nuevos])

#print(list[df_concatenated_2.columns])




df_concatenated_2['RUT'] = df_concatenated_2['RUT'].astype(str)
resultado['RUT'] = resultado['RUT'].astype(str)
# Fusionar DataFrame_2 con el DataFrame de resultado utilizando la columna 'RUT' como clave de fusión

dataframe_final = df_concatenated_2.merge(resultado, on='RUT', how='left')

#print(list[dataframe_final.columns])


dataframe_final_2 = crear_dataframe_ultima_atencion(dataframe_final)

In [56]:
df_concatenated_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion,DIAGNOSTICOS,Telefono Consolidado2
0,Cancer de Pulmón,5513331,K,MARÍA MAGALY MIRANDA VÁSQUEZ,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor maligno de los bronquios y del pulmon'],930854428/NO ESPECIFICADO/-,NaN,NaN,24-06-2024,Médico JOSE RAMON SILVA FIGUEROA Derivacion de...,NaN,NaN
0,Cáncer de Próstata,10467223,K,NAIM ANTONIO SALVADOR CHOMALÍ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de la prostata],-/-/-
1,Cáncer de Próstata,10754884,K,MIGUEL ANGEL ORTIZ SOLORZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de la prostata, nan]",///-/-
2,Cáncer de Próstata,10961756,3,WILSON MIGUEL VERDUGO CERDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[CÁNCER DE PRÓSTATA ETAPA IV],-/-/-
3,Cáncer de Próstata,11654329,K,PATRICIO ENRIQUE CABRERA ALVARADO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA./MTT ESPINAL],-/-/-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Cáncer de Próstata,9233670,0,JORGE ARTURO PEÑA RIFFO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[CANCER DE PROSTATA],-/-/-
109,Cáncer de Próstata,9257543,8,JUAN IGNACIO VÁSQUEZ LEIVA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA],-/-/-
110,Cáncer de Próstata . {decreto nº 228},9270788,1,"BRAVIN HERRERA, GIORGIO ALDO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 26131454 / 0 / 57602865 / 22553514 / -...
111,Cáncer de Próstata,9384687,7,RODOLFO BRUNO PEDRAZA CORROTEA,Alta,Fallecido,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA ],-/-/-


In [57]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,5513331,NaT,NaN
1,10467223,2024-05-07 12:05:00,Médico JORGE HUGO ALMONACID GRUNERT Control - ...
2,10754884,NaT,NaN
3,10961756,2024-01-11 14:42:00,Médico MAXIMILIANO EUGENIO NARVÁEZ FUENTES Con...
4,11654329,2023-03-30 14:04:00,Médico JORGE HUGO ALMONACID GRUNERT Control po...
...,...,...,...
108,9233670,NaT,NaN
109,9257543,NaT,NaN
110,9270788,2024-07-08 11:30:00,Médico VÍCTOR MANUEL AVILLO CAROCA Control - C...
111,9384687,NaT,NaN


In [58]:
# Convertir la columna de fecha a formato datetime
dataframe_final_2['Fecha Ultima atencion'] = pd.to_datetime(dataframe_final_2['Fecha Ultima atencion'])

# Formatear la columna de fecha en el formato deseado
dataframe_final_2['Fecha Ultima atencion'] = dataframe_final_2['Fecha Ultima atencion'].dt.strftime('%Y-%m-%d')


In [59]:
dataframe_final_2 = dataframe_final_2.dropna()

In [60]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
1,10467223,2024-05-07,Médico JORGE HUGO ALMONACID GRUNERT Control - ...
3,10961756,2024-01-11,Médico MAXIMILIANO EUGENIO NARVÁEZ FUENTES Con...
4,11654329,2023-03-30,Médico JORGE HUGO ALMONACID GRUNERT Control po...
5,21903974,2024-03-27,Médico PABLO ALFREDO AROCA SIRE Control - Cons...
6,2614835,2022-04-08,Médico JOAN CRISTÓBAL BASSA MOYANO Control - C...
...,...,...,...
105,9034836,2024-07-02,Médico JOSÉ FRANCISCO LÓPEZ JOANNON Derivacion...
106,9069762,2024-05-09,Médico JORGE HUGO ALMONACID GRUNERT CONSULTA A...
107,9157681,2024-01-02,Médico PABLO ALFREDO AROCA SIRE Control - Cons...
110,9270788,2024-07-08,Médico VÍCTOR MANUEL AVILLO CAROCA Control - C...


In [61]:
# Convertir la columna de fecha a formato datetime
dataframe_final_2['Fecha Ultima atencion'] = pd.to_datetime(dataframe_final_2['Fecha Ultima atencion'])

# Formatear la columna de fecha en el formato deseado
dataframe_final_2['Fecha Ultima atencion'] = dataframe_final_2['Fecha Ultima atencion'].dt.strftime('%d-%m-%Y')


In [62]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
1,10467223,07-05-2024,Médico JORGE HUGO ALMONACID GRUNERT Control - ...
3,10961756,11-01-2024,Médico MAXIMILIANO EUGENIO NARVÁEZ FUENTES Con...
4,11654329,30-03-2023,Médico JORGE HUGO ALMONACID GRUNERT Control po...
5,21903974,27-03-2024,Médico PABLO ALFREDO AROCA SIRE Control - Cons...
6,2614835,08-04-2022,Médico JOAN CRISTÓBAL BASSA MOYANO Control - C...
...,...,...,...
105,9034836,02-07-2024,Médico JOSÉ FRANCISCO LÓPEZ JOANNON Derivacion...
106,9069762,09-05-2024,Médico JORGE HUGO ALMONACID GRUNERT CONSULTA A...
107,9157681,02-01-2024,Médico PABLO ALFREDO AROCA SIRE Control - Cons...
110,9270788,08-07-2024,Médico VÍCTOR MANUEL AVILLO CAROCA Control - C...


In [63]:
import pandas
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [64]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(dataframe_final_2, index= False, header = True):
    ws.append(r)

for cell in ws['B'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_ultima_atencion_prostata.xlsx'
wb.save(os.path.join(dir_actual, '..','CA UROLOGICOS',f'{nombre_archivo}'))

# Guardar dataframe ultima atención 

In [65]:
#nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_ultima_atencion_tiroides.xlsx'  # Nombre del archivo de Excel
#dataframe_final_2.to_excel(os.path.join(dir_actual, '..','CA TIROIDES',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 


In [66]:
df1 = df_concatenated_2.copy()
df2 = dataframe_final_2.copy()

df1['RUT'] = df1['RUT'].astype(str)
df2['RUT'] = df2['RUT'].astype(str)


# Realizar la fusión utilizando la columna 'RUT'
df_merged = pd.merge(df1, df2, on='RUT', how='left')



In [67]:
df2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
1,10467223,07-05-2024,Médico JORGE HUGO ALMONACID GRUNERT Control - ...
3,10961756,11-01-2024,Médico MAXIMILIANO EUGENIO NARVÁEZ FUENTES Con...
4,11654329,30-03-2023,Médico JORGE HUGO ALMONACID GRUNERT Control po...
5,21903974,27-03-2024,Médico PABLO ALFREDO AROCA SIRE Control - Cons...
6,2614835,08-04-2022,Médico JOAN CRISTÓBAL BASSA MOYANO Control - C...
...,...,...,...
105,9034836,02-07-2024,Médico JOSÉ FRANCISCO LÓPEZ JOANNON Derivacion...
106,9069762,09-05-2024,Médico JORGE HUGO ALMONACID GRUNERT CONSULTA A...
107,9157681,02-01-2024,Médico PABLO ALFREDO AROCA SIRE Control - Cons...
110,9270788,08-07-2024,Médico VÍCTOR MANUEL AVILLO CAROCA Control - C...


In [68]:
len(df1['RUT'].unique())

113

In [69]:
len(df2['RUT'].unique())

70

In [70]:
# Renombrar columnas para evitar colisiones
df_merged.rename(columns={'Fecha Ultima atencion_y': 'Fecha Ultima atencion', 'Profesional Ultima atencion_y': 'Profesional Ultima atencion'}, inplace=True)

# Eliminar columnas duplicadas
df_merged.drop(['Fecha Ultima atencion_x', 'Profesional Ultima atencion_x'], axis=1, inplace=True)



In [71]:
df_merged[df_merged.duplicated(subset=['RUT'], keep=False)]

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,DIAGNOSTICOS,Telefono Consolidado2,Fecha Ultima atencion,Profesional Ultima atencion


In [72]:
df_merged = df_merged.drop_duplicates(subset=['RUT'], keep='first')


In [73]:
df_merged

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,DIAGNOSTICOS,Telefono Consolidado2,Fecha Ultima atencion,Profesional Ultima atencion
0,Cancer de Pulmón,5513331,K,MARÍA MAGALY MIRANDA VÁSQUEZ,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor maligno de los bronquios y del pulmon'],930854428/NO ESPECIFICADO/-,NaN,NaN,NaN,NaN,NaN,NaN
1,Cáncer de Próstata,10467223,K,NAIM ANTONIO SALVADOR CHOMALÍ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Tumor maligno de la prostata],-/-/-,07-05-2024,Médico JORGE HUGO ALMONACID GRUNERT Control - ...
2,Cáncer de Próstata,10754884,K,MIGUEL ANGEL ORTIZ SOLORZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno de la prostata, nan]",///-/-,NaN,NaN
3,Cáncer de Próstata,10961756,3,WILSON MIGUEL VERDUGO CERDA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[CÁNCER DE PRÓSTATA ETAPA IV],-/-/-,11-01-2024,Médico MAXIMILIANO EUGENIO NARVÁEZ FUENTES Con...
4,Cáncer de Próstata,11654329,K,PATRICIO ENRIQUE CABRERA ALVARADO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA./MTT ESPINAL],-/-/-,30-03-2023,Médico JORGE HUGO ALMONACID GRUNERT Control po...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Cáncer de Próstata,9233670,0,JORGE ARTURO PEÑA RIFFO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[CANCER DE PROSTATA],-/-/-,NaN,NaN
109,Cáncer de Próstata,9257543,8,JUAN IGNACIO VÁSQUEZ LEIVA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA],-/-/-,NaN,NaN
110,Cáncer de Próstata . {decreto nº 228},9270788,1,"BRAVIN HERRERA, GIORGIO ALDO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 26131454 / 0 / 57602865 / 22553514 / -...,08-07-2024,Médico VÍCTOR MANUEL AVILLO CAROCA Control - C...
111,Cáncer de Próstata,9384687,7,RODOLFO BRUNO PEDRAZA CORROTEA,Alta,Fallecido,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[CA PROSTATA ],-/-/-,NaN,NaN


# Crear Citas futuras

In [74]:

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)



ruta_del_archivo = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','AgendasFuturas', f'{today.year}{month_formatted}{day_formatted}_Citas_Futuras.csv')


# Carga el archivo XLSX en un DataFrame
dataframe_fu = cargar_dataframe_ambulatorio(ruta_del_archivo)

In [75]:
# Mover los nombres de las columnas una posición hacia la izquierda
new_columns = list(dataframe_fu.columns)
new_columns = new_columns[1:] + [new_columns[0]]
dataframe_fu.columns = new_columns


In [76]:

dataframe_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1720638534265700951,EU. NATALY ESTEFANÍA ÁLVAREZ BASAURE,Enfermería Multiespecialidad,12-07-2024,1030,Agendada,Curación Avanzada,CEX,N,N,NaN,...,956384216,NaN,NaN,NaN,LOS ALACALUFES,La Florida,956384216,NaN,NaN,NaN
1720638534265700951,EU. NATALY ESTEFANÍA ÁLVAREZ BASAURE,Enfermería Multiespecialidad,12-07-2024,1030,Agendada,Curación Avanzada,CEX,N,N,NaN,...,956384216,NaN,NaN,NaN,LOS ALACALUFES,La Florida,956384216,NaN,NaN,NaN
1719248205024500571,Becados DR. Zeyuan Zhang,Cirugía Adulto,12-07-2024,1300,Agendada,Control postquirurgico,CEX,N,N,NaN,...,965821706,NaN,NaN,NaN,ventisquero del museo,Puente Alto,965821706,NaN,NaN,NaN
1719248205024500571,Becados DR. Zeyuan Zhang,Cirugía Adulto,12-07-2024,1300,Agendada,Control postquirurgico,CEX,N,N,NaN,...,965821706,NaN,NaN,NaN,ventisquero del museo,Puente Alto,965821706,NaN,NaN,NaN
1719594036816500140,DR. JOAQUÍN NICOLÁS ANANÍAS ZAROR,Traumatología Adulto,12-07-2024,1000,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,984237786,NaN,NaN,NaN,LAS CODORNICES,La Florida,984237786,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1719594283483500202,EU. SAMANTHA SOFÍA JORQUERA DROGUETT,Enfermería Multiespecialidad,27-12-2024,1200,Agendada,Control Hormonoterapia,CEX,N,N,NaN,...,940091392,02095019583,37812474,NaN,Corindon,La Florida,940091392,02095019583,37812474,NaN
1720629658078700729,TOMA DE MUESTRA,Enfermería,27-12-2024,900,Agendada,Genérica toma de muestras,CEX,N,N,NaN,...,NaN,32080831,NaN,NaN,Pje 9,La Florida,NaN,32080831,NaN,NaN
1719594986790500330,EU. SAMANTHA SOFÍA JORQUERA DROGUETT,Enfermería Multiespecialidad,27-12-2024,1230,Agendada,Control Hormonoterapia,CEX,N,N,NaN,...,0998422105,0225180285,NaN,NaN,LOS CLARINES,La Florida,0998422105,0225180285,NaN,NaN
1719949122581710946,TOMA DE MUESTRA,Enfermería,30-12-2024,1000,Agendada,Genérica toma de muestras,CEX,N,N,NaN,...,56992967022,56992967022,56992967022,NaN,Santa Cecilia,La Florida,56992967022,56992967022,56992967022,NaN


In [77]:
# Eliminar duplicados por índice y mantener el primero
df_sin_duplicados = dataframe_fu.drop_duplicates(keep='first')


In [78]:
df_sin_duplicados['ESTADO_CITA']

1720638534265700951    Agendada
1719248205024500571    Agendada
1719594036816500140    Agendada
1719258604708510192    Agendada
1720473502007700950    Agendada
                         ...   
1719595680916500422    Agendada
1719594986790500330    Agendada
1720629658078700729    Agendada
1719594283483500202    Agendada
1719949122581710946    Agendada
Name: ESTADO_CITA, Length: 29547, dtype: object

In [79]:
df_sin_duplicados['ESTADO_CITA'].unique()

array(['Agendada', 'Anulada', 'En buzón de reasignación'], dtype=object)

In [80]:
df_sin_duplicados[df_sin_duplicados['ESPECIALIDAD_AGENDA']=='Imagenología AMB']['DESCRIPCION']

1720646994802700707                    Ecografia Cerebral en RN
1720451499484710103                        Rx Antebrazo Der F-L
1720614007641710683                         Rx Muñeca Izq F-L-O
1718377032563510924                              Rx Pie Der F-L
1718376997451510916                              Rx Pie Izq F-L
                                         ...                   
1719950355682710292                             Rx Codo Der F-L
1712323640596910122                    Rx Cadera Der Crosstable
1712323464883910069    Rx Pelvis AP para evaluación de Prótesis
1716830805648300587                                Rx Pelvis AP
1703877427182900912               TC de Abdomen y Pelvis con MC
Name: DESCRIPCION, Length: 1264, dtype: object

In [81]:
list(df_sin_duplicados)

['NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'CITAHORA',
 'ESTADO_CITA',
 'DESCRIPCION',
 'ORIGEN',
 'GES_CITA',
 'GES_SOL',
 'PATOLOGIA_GES_SOL',
 'PACIENTE',
 'CITA_CODUTEN',
 'DOCUMENTO',
 'NOMBRE_TIPO_DOCUMENTO',
 'NUM_DOCUMENTO_TUTOR',
 'NOMBRE_DOCUMENTO_TUTOR',
 'CITAPACIFECHNACI',
 'SEXO',
 'NACIONALIDAD',
 'FACT_FINANCIADOR',
 'FACT_COBERTURA',
 'GES_EPISODIO',
 'MOVIL_HABITUAL',
 'TELEFONO1_HABITUAL',
 'TELEFONO2_HABITUAL',
 'TELEFONO3_HABITUALDIRECCION_HABITUAL',
 'COMUNA_HABITUAL',
 'MOVIL_CONTACTO',
 'TELEFONO1_CONTACTO',
 'TELEFONO2_CONTACTO',
 'TELEFONO3_CONTACTO',
 'CITA_KEY']

In [82]:
# Filtrar por las especialidades de Endocrinología Adulto y Endocrinología Infantil
endocrinologia_df_fu = df_sin_duplicados[df_sin_duplicados['ESPECIALIDAD_AGENDA'].isin(['Urología Adulto','Imagenología AMB'])]
    # Filtrar las filas que no contienen 'No realizada' o 'Anulada' en la columna 'ESTADO_CITA'
endocrinologia_df_fu = endocrinologia_df_fu[~endocrinologia_df_fu['ESTADO_CITA'].isin(['Anulada', 'En buzón de reasignación'])]

In [83]:
list(endocrinologia_df_fu.columns)

['NOMBRE_AGENDA',
 'ESPECIALIDAD_AGENDA',
 'CITAFECH',
 'CITAHORA',
 'ESTADO_CITA',
 'DESCRIPCION',
 'ORIGEN',
 'GES_CITA',
 'GES_SOL',
 'PATOLOGIA_GES_SOL',
 'PACIENTE',
 'CITA_CODUTEN',
 'DOCUMENTO',
 'NOMBRE_TIPO_DOCUMENTO',
 'NUM_DOCUMENTO_TUTOR',
 'NOMBRE_DOCUMENTO_TUTOR',
 'CITAPACIFECHNACI',
 'SEXO',
 'NACIONALIDAD',
 'FACT_FINANCIADOR',
 'FACT_COBERTURA',
 'GES_EPISODIO',
 'MOVIL_HABITUAL',
 'TELEFONO1_HABITUAL',
 'TELEFONO2_HABITUAL',
 'TELEFONO3_HABITUALDIRECCION_HABITUAL',
 'COMUNA_HABITUAL',
 'MOVIL_CONTACTO',
 'TELEFONO1_CONTACTO',
 'TELEFONO2_CONTACTO',
 'TELEFONO3_CONTACTO',
 'CITA_KEY']

In [84]:
# Convertir la columna de fecha a formato datetime
#endocrinologia_df_fu['CITAFECH'] = pd.to_datetime(endocrinologia_df_fu['CITAFECH'])

# Formatear la columna de fecha en el formato deseado
#endocrinologia_df_fu['CITAFECH'] = endocrinologia_df_fu['CITAFECH'].dt.strftime('%Y-%m-%d')

In [85]:
endocrinologia_df_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1720646994802700707,ECO I2-2,Imagenología AMB,12-07-2024,1000,Agendada,Ecografia Cerebral en RN,CEX,N,N,NaN,...,50024563,NO ESPECIFICADO,NaN,NaN,kon tiki,Puente Alto,50024563,NO ESPECIFICADO,NaN,NaN
1720451499484710103,Sala I-3,Imagenología AMB,12-07-2024,900,Agendada,Rx Antebrazo Der F-L,CEX,N,N,NaN,...,0998790044,NaN,NaN,NaN,PEUTREN CENTRAL,San Ramón,0998790044,NaN,NaN,NaN
1720614007641710683,Sala I-5,Imagenología AMB,12-07-2024,800,Agendada,Rx Muñeca Izq F-L-O,CEX,N,N,NaN,...,974098559,NaN,NaN,NaN,cezanne,La Florida,974098559,NaN,NaN,NaN
1718377032563510924,Sala I-4,Imagenología AMB,12-07-2024,1540,Agendada,Rx Pie Der F-L,CEX,N,N,NaN,...,56979563424,56988199139,NaN,NaN,COLOMBIA,La Florida,56979563424,56988199139,NaN,NaN
1718376997451510916,Sala I-4,Imagenología AMB,12-07-2024,1530,Agendada,Rx Pie Izq F-L,CEX,N,N,NaN,...,56979563424,56988199139,NaN,NaN,COLOMBIA,La Florida,56979563424,56988199139,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711570784927900514,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1545,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,56998623844,56988586700,NaN,NaN,Guacolda,La Florida,56998623844,56988586700,NaN,NaN
1711981070379900773,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1415,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,0987243463,02225166822,966980845,NaN,Pleamar,La Florida,0987243463,02225166822,966980845,NaN
1711548687149910674,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1400,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,74879538,74879538,NaN,NaN,rio pudeto,Puente Alto,74879538,74879538,NaN,NaN
1716830805648300587,Sala I-3,Imagenología AMB,12-11-2024,1100,Agendada,Rx Pelvis AP,CEX,N,N,NaN,...,090000000000,NaN,NaN,NaN,NaN,La Florida,090000000000,NaN,NaN,NaN


In [86]:
endocrinologia_df_fu['DV'] = endocrinologia_df_fu['DOCUMENTO'].str[-1]

endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['DOCUMENTO'].str[:-1]

In [87]:
endocrinologia_df_fu['NOMBRE_TIPO_DOCUMENTO']

1720646994802700707    RUN
1720451499484710103    RUN
1720614007641710683    RUN
1718377032563510924    RUN
1718376997451510916    RUN
                      ... 
1711570784927900514    RUN
1711981070379900773    RUN
1711548687149910674    RUN
1716830805648300587    RUN
1703877427182900912    RUN
Name: NOMBRE_TIPO_DOCUMENTO, Length: 2467, dtype: object

In [88]:
endocrinologia_df_fu = endocrinologia_df_fu[endocrinologia_df_fu['NOMBRE_TIPO_DOCUMENTO']=='RUN']

In [89]:
endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['RUT'].astype(int)

In [90]:
endocrinologia_df_fu = endocrinologia_df_fu[['RUT','NOMBRE_AGENDA','ESPECIALIDAD_AGENDA','CITAFECH','CITAHORA','ESTADO_CITA','DESCRIPCION','ORIGEN','GES_CITA','GES_SOL','PATOLOGIA_GES_SOL','PACIENTE','CITA_CODUTEN','DOCUMENTO']]

In [91]:
endocrinologia_df_fu

,RUT,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,PACIENTE,CITA_CODUTEN,DOCUMENTO
1720646994802700707,28433687,ECO I2-2,Imagenología AMB,12-07-2024,1000,Agendada,Ecografia Cerebral en RN,CEX,N,N,NaN,VICENTE FACUNDO QUINTANILLA TORRES,AC00549028,284336879
1720451499484710103,26874276,Sala I-3,Imagenología AMB,12-07-2024,900,Agendada,Rx Antebrazo Der F-L,CEX,N,N,NaN,AVRIL FLORENCIA GARÍN TORRES,AC00549829,26874276K
1720614007641710683,24237500,Sala I-5,Imagenología AMB,12-07-2024,800,Agendada,Rx Muñeca Izq F-L-O,CEX,N,N,NaN,MIGUEL ANGEL JOSÉ GONZÁLEZ FUENTES,AC00550248,242375009
1718377032563510924,8380494,Sala I-4,Imagenología AMB,12-07-2024,1540,Agendada,Rx Pie Der F-L,CEX,N,N,NaN,ROSSE MARY RODRÍGUEZ AVENDAÑO,AC00546438,83804947
1718376997451510916,8380494,Sala I-4,Imagenología AMB,12-07-2024,1530,Agendada,Rx Pie Izq F-L,CEX,N,N,NaN,ROSSE MARY RODRÍGUEZ AVENDAÑO,AC00546438,83804947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711570784927900514,3727939,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1545,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,JOSÉ MARCELO CASTILLO LAVOZ,AC00234789,37279390
1711981070379900773,7893201,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1415,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,SERGIO JORGE CRESPO HERNÁNDEZ,AC00240804,78932015
1711548687149910674,6849972,DR. RODRIGO ANTONIO GONZÁLEZ IRRIBARRA,Urología Adulto,24-10-2024,1400,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,LUIS SEREY VEGA,AC00041717,68499720
1716830805648300587,28119064,Sala I-3,Imagenología AMB,12-11-2024,1100,Agendada,Rx Pelvis AP,CEX,N,N,NaN,BENJAMIN EXEQUIEL JARA CLAVERIA,AC00519708,281190644


In [92]:
# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)


In [93]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(endocrinologia_df_fu, index= False, header = True):
    ws.append(r)

for cell in ws['D'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_citas_futuras_prostata.xlsx'
wb.save(os.path.join(dir_actual, '..','CA UROLOGICOS',f'{nombre_archivo}'))